In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
import itertools

In [2]:
path = '../datasets/newyork-stock-exchange/RedditNews.csv'
data = pd.read_csv(path)
dates = data['Date']
news = data['News']
daily_news = data.groupby("Date")['News'].apply('<.>'.join).reset_index()
print(daily_news.shape)
print(daily_news['Date'].head(5))

(2943, 2)
0    2008-06-08
1    2008-06-09
2    2008-06-10
3    2008-06-11
4    2008-06-12
Name: Date, dtype: object


In [3]:
path = '../datasets/newyork-stock-exchange/upload_DJIA_table.csv'
stock_data = pd.read_csv(path)
date = pd.DataFrame(stock_data['Date'])
open_value = stock_data['Open']
close_value = stock_data['Close']
result = pd.DataFrame(close_value >= open_value, columns=['Label'])
market_trend = date.join(result)
print(market_trend.shape)
print(market_trend.tail(5))

(1989, 2)
            Date  Label
1984  2008-08-14   True
1985  2008-08-13  False
1986  2008-08-12  False
1987  2008-08-11   True
1988  2008-08-08   True


In [4]:
merged_data = pd.merge(daily_news, market_trend, on='Date')
merged_data = merged_data[['Date', 'Label', 'News']]
print(merged_data.shape)
print(merged_data.head(5))

(1989, 3)
         Date  Label                                               News
0  2008-08-08   True  b"Georgia 'downs two Russian warplanes' as cou...
1  2008-08-11   True  b'Why wont America and Nato help us? If they w...
2  2008-08-12  False  b'Remember that adorable 9-year-old who sang a...
3  2008-08-13  False  b' U.S. refuses Israel weapons to attack Iran:...
4  2008-08-14   True  b'All the experts admit that we should legalis...


In [8]:
path = '../datasets/newyork-stock-exchange/combined-news-stock.txt'
merged_data.to_csv(path, sep='\t', index=False)

In [9]:
path = '../datasets/newyork-stock-exchange/combined-news-stock.txt'
data = pd.read_csv(path, sep="\t")
# date = data['Date']
# label = data['Label']
# news = data['News']
all_news = {}
for index, row in data.iterrows():
    news = row['News']
    date = row['Date']
    headlines = news.split("<.>")
    daily_headlines = []
    for headline in headlines:
        headline = headline.strip('"b')
        #converting the sentences into words
        tokens = word_tokenize(headline)
        #converting to lowercas
        tokens = [w.lower() for w in tokens]
        #remvoing non english alphabetic character
        words = [word for word in tokens if word.isalpha()]
        #removing the stop words
        #words = [w for w in words if not w in stop_words]
        #print(words)
        daily_headlines.append(words)
        all_news[date] = daily_headlines
        
for n in all_news:
    print(all_news[n])
    break

[['georgia', 'two', 'russian', 'warplanes', 'as', 'countries', 'move', 'to', 'brink', 'of', 'war'], ['musharraf', 'to', 'be', 'impeached'], ['today', 'columns', 'of', 'troops', 'roll', 'into', 'south', 'ossetia', 'footage', 'from', 'fighting', 'youtube'], ['tanks', 'are', 'moving', 'towards', 'the', 'capital', 'of', 'south', 'ossetia', 'which', 'has', 'reportedly', 'been', 'completely', 'destroyed', 'by', 'georgian', 'artillery', 'fire'], ['afghan', 'children', 'raped', 'with', 'official', 'says', 'this', 'is', 'sick', 'a', 'three', 'year', 'old', 'was', 'raped', 'and', 'they', 'do', 'nothing'], ['russian', 'tanks', 'have', 'entered', 'south', 'ossetia', 'whilst', 'georgia', 'shoots', 'down', 'two', 'russian', 'jets'], ['breaking', 'georgia', 'invades', 'south', 'ossetia', 'russia', 'warned', 'it', 'would', 'intervene', 'on', 'so', 'side'], ['the', 'combatent', 'trials', 'are', 'nothing', 'but', 'a', 'sham', 'salim', 'haman', 'has', 'been', 'sentenced', 'to', 'years', 'but', 'will', 'b

In [7]:
def dictionary(raw_news,threshold):
    caps = []
    for date in raw_news:
        for news in raw_news[date]:
            caps.append(news)
    
    word_freq = nltk.FreqDist(itertools.chain(*caps))
    id_to_word = ['<pad>'] + [word for word, cnt in word_freq.items() if cnt >= threshold] + ['<unk>']
    word_to_id = {word:idx for idx, word in enumerate(id_to_word)}
    
    return id_to_word, word_to_id

id_to_word, word_to_id = dictionary(all_news, 4)
print(word_to_id)

{'<pad>': 0, 'georgia': 1, 'two': 2, 'russian': 3, 'warplanes': 4, 'as': 5, 'countries': 6, 'move': 7, 'to': 8, 'brink': 9, 'of': 10, 'war': 11, 'musharraf': 12, 'be': 13, 'today': 14, 'troops': 15, 'roll': 16, 'into': 17, 'south': 18, 'ossetia': 19, 'footage': 20, 'from': 21, 'fighting': 22, 'youtube': 23, 'tanks': 24, 'are': 25, 'moving': 26, 'towards': 27, 'the': 28, 'capital': 29, 'which': 30, 'has': 31, 'reportedly': 32, 'been': 33, 'completely': 34, 'destroyed': 35, 'by': 36, 'georgian': 37, 'artillery': 38, 'fire': 39, 'afghan': 40, 'children': 41, 'raped': 42, 'with': 43, 'official': 44, 'says': 45, 'this': 46, 'is': 47, 'sick': 48, 'a': 49, 'three': 50, 'year': 51, 'old': 52, 'was': 53, 'and': 54, 'they': 55, 'do': 56, 'nothing': 57, 'have': 58, 'entered': 59, 'whilst': 60, 'shoots': 61, 'down': 62, 'jets': 63, 'breaking': 64, 'invades': 65, 'russia': 66, 'warned': 67, 'it': 68, 'would': 69, 'intervene': 70, 'on': 71, 'so': 72, 'side': 73, 'trials': 74, 'but': 75, 'sham': 76, 